In [5]:
SELECT * FROM Gold_LH.dbo.dim_customers LIMIT 1000

StatementMeta(, b72f812e-edbe-47f3-996d-3a47edfcc6df, 6, Finished, Available, Finished)

<Spark SQL result set with 200 rows and 8 fields>

**Question 1a, Driver performance**: KPI for ( On-time rates, MPG, revenue per mile )


In [14]:
WITH cte_overall_kpis AS (

    SELECT
        /* Total Revenue */
        COALESCE(sub3.total_revenue, 0) AS total_revenue,

        /* Overall On-Time Delivery Rate (2 decimals) */
        ROUND(
            CASE
                WHEN COALESCE(sub1.total_deliveries, 0) = 0 THEN NULL
                ELSE COALESCE(sub1.on_time_deliveries, 0) * 1.0 / sub1.total_deliveries
            END,
            2
        ) AS on_time_delivery_rate,

        /* Avg MPG (2 decimals) */
        ROUND(
            CASE
                WHEN COALESCE(sub2.total_fuel_gallons, 0) = 0 THEN NULL
                ELSE COALESCE(sub2.total_miles, 0) * 1.0 / sub2.total_fuel_gallons
            END,
            2
        ) AS avg_mpg,

        /* Avg Revenue per Mile (2 decimals) */
        ROUND(
            CASE
                WHEN COALESCE(sub2.total_miles, 0) = 0 THEN NULL
                ELSE COALESCE(sub3.total_revenue, 0) * 1.0 / sub2.total_miles
            END,
            2
        ) AS avg_revenue_per_mile

    FROM (SELECT 1 AS k) t1

    /* sub1 = deliveries overall (via trips) */
    LEFT JOIN (
        SELECT
            COUNT(t2.event_id) AS total_deliveries,
            SUM(CASE WHEN t2.on_time_flag = 1 THEN 1 ELSE 0 END) AS on_time_deliveries
        FROM Gold_LH.dbo.fact_delivery_events t2
        -- If you only want delivered events, uncomment and adjust:
        -- WHERE UPPER(t2.event_type) = 'DELIVERY'
    ) sub1
        ON sub1.total_deliveries IS NOT NULL

    /* sub2 = miles + fuel overall */
    LEFT JOIN (
        SELECT
            SUM(COALESCE(t2.actual_distance_miles, 0)) AS total_miles,
            SUM(COALESCE(t2.fuel_gallons_used, 0))     AS total_fuel_gallons
        FROM Gold_LH.dbo.fact_trips t2
    ) sub2
        ON sub2.total_miles IS NOT NULL

    /* sub3 = revenue overall */
    LEFT JOIN (
        SELECT
            SUM(COALESCE(t2.total_revenue, 0)) AS total_revenue
        FROM Gold_LH.dbo.fact_driver_monthly_metrics t2
    ) sub3
        ON sub3.total_revenue IS NOT NULL
)

SELECT
    t1.on_time_delivery_rate,
    t1.avg_mpg,
    t1.avg_revenue_per_mile,
    t1.total_revenue
FROM cte_overall_kpis t1;


StatementMeta(, 510ae018-b510-4460-84f4-d292be473f0b, 17, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

**Question 1a, Driver Performance**: This query provides a high-level operational view with driver-level granularity by revenue per mile ranking

In [13]:
WITH cte_driver_kpis AS (

    SELECT
        t1.driver_id,

        /* Deliveries */
        COALESCE(sub1.total_deliveries, 0)   AS total_deliveries,
        COALESCE(sub1.on_time_deliveries, 0) AS on_time_deliveries,

        /* On-Time Delivery Rate (2 decimals) */
        ROUND(
            CASE
                WHEN COALESCE(sub1.total_deliveries, 0) = 0 THEN NULL
                ELSE COALESCE(sub1.on_time_deliveries, 0) * 1.0 / sub1.total_deliveries
            END,
            2
        ) AS on_time_delivery_rate,

        /* Miles + Fuel */
        COALESCE(sub2.total_miles, 0) AS total_miles,

        /* Total Fuel Gallons (2 decimals) */
        ROUND(COALESCE(sub2.total_fuel_gallons, 0), 2) AS total_fuel_gallons,

        /* Driver MPG (2 decimals) */
        ROUND(
            CASE
                WHEN COALESCE(sub2.total_fuel_gallons, 0) = 0 THEN NULL
                ELSE COALESCE(sub2.total_miles, 0) * 1.0 / sub2.total_fuel_gallons
            END,
            2
        ) AS driver_mpg,

        /* Revenue */
        COALESCE(sub3.total_revenue, 0) AS total_revenue,

        /* Avg Revenue per Mile (2 decimals) */
        ROUND(
            CASE
                WHEN COALESCE(sub2.total_miles, 0) = 0 THEN NULL
                ELSE COALESCE(sub3.total_revenue, 0) * 1.0 / sub2.total_miles
            END,
            2
        ) AS avg_revenue_per_mile

    FROM Gold_LH.dbo.dim_drivers t1

    /* sub1 = deliveries by driver */
    LEFT JOIN (
        SELECT
            t2.driver_id,
            COUNT(t3.event_id) AS total_deliveries,
            SUM(CASE WHEN t3.on_time_flag = 1 THEN 1 ELSE 0 END) AS on_time_deliveries
        FROM Gold_LH.dbo.fact_trips t2
        JOIN Gold_LH.dbo.fact_delivery_events t3
            ON t3.trip_id = t2.trip_id
        GROUP BY
            t2.driver_id
    ) sub1
        ON sub1.driver_id = t1.driver_id

    /* sub2 = miles and fuel */
    LEFT JOIN (
        SELECT
            t2.driver_id,
            SUM(COALESCE(t2.actual_distance_miles, 0)) AS total_miles,
            SUM(COALESCE(t2.fuel_gallons_used, 0))     AS total_fuel_gallons
        FROM Gold_LH.dbo.fact_trips t2
        GROUP BY
            t2.driver_id
    ) sub2
        ON sub2.driver_id = t1.driver_id

    /* sub3 = revenue */
    LEFT JOIN (
        SELECT
            driver_id,
            SUM(COALESCE(total_revenue, 0)) AS total_revenue
        FROM Gold_LH.dbo.fact_driver_monthly_metrics
        GROUP BY
            driver_id
    ) sub3
        ON sub3.driver_id = t1.driver_id
)

SELECT
    t1.driver_id,
    t1.total_deliveries,
    t1.on_time_deliveries,
    t1.on_time_delivery_rate,
   --t1.total_miles,
 --  t1.total_fuel_gallons,
    t1.driver_mpg,
   DENSE_RANK() OVER (ORDER BY t1.driver_mpg DESC NULLS LAST) AS mpg_rank,
   t1.total_revenue,
    t1.avg_revenue_per_mile,
    DENSE_RANK() OVER (ORDER BY t1.avg_revenue_per_mile DESC NULLS LAST) AS revenue_per_mile_rank
FROM cte_driver_kpis t1
ORDER BY
    revenue_per_mile_rank,
  mpg_rank,
   t1.driver_id;


StatementMeta(, 510ae018-b510-4460-84f4-d292be473f0b, 16, Finished, Available, Finished)

<Spark SQL result set with 150 rows and 9 fields>

**Q-1B, Driver Monthly Performance** starting from January 2024, using the pre-aggregated Gold-layer table fact_driver_monthly_metrics.

In [4]:
SELECT
    t1.driver_id                                    AS `Driver ID`,
    t1.month                                        AS `Date`,
    ROUND(t1.on_time_delivery_rate * 100, 2)        AS `Delivery rate`,
    ROUND(t1.average_idle_hours, 2)                 AS `Avg Idle Hours`,
    ROUND(t1.average_mpg, 2)                        AS `Average MPG`,
    CAST(t1.trips_completed AS BIGINT)              AS `Trips Completed`,
    ROUND(t1.total_miles, 0)                        AS `Total Miles`,
    ROUND(t1.total_revenue, 0)                      AS `Total Revenue`
FROM Gold_LH.dbo.fact_driver_monthly_metrics t1
WHERE t1.driver_id = 'DRV00001' AND t1.month >= DATE '2024-01-01'
ORDER BY
    t1.driver_id,
    t1.month;


StatementMeta(, 08eb05d1-9e74-482a-ae26-137968f91534, 6, Finished, Available, Finished)

<Spark SQL result set with 12 rows and 8 fields>

In [7]:
class(df)


StatementMeta(, 08eb05d1-9e74-482a-ae26-137968f91534, 9, Finished, Available, Finished)

Error: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'class'.(line 1, pos 0)

== SQL ==
class(df)
^^^


# **Switching to Spark R for Visualization.**
The previous cells used Spark SQL to prepare and aggregate logistics performance data in the Gold layer.
This section switches to Spark R to leverage R’s visualization libraries for time-series trend analysis.

In [10]:
library(SparkR)
library(ggplot2)
library(scales)

# 1) Pull data from your Gold table (Spark SQL)
sdf <- sql("
  SELECT
    month,
    total_miles,
    average_mpg
  FROM Gold_LH.dbo.fact_driver_monthly_metrics
WHERE t1.driver_id = 'DRV00001' AND t1.month >= DATE '2024-01-01'
  ORDER BY month
")

# 2) Convert Spark DataFrame -> R data.frame
pdf <- collect(sdf)

# 3) Ensure month is a Date type in R
pdf$month <- as.Date(pdf$month)

# ---- Chart 1: Monthly Miles Trend ----
p_miles <- ggplot(pdf, aes(x = month, y = total_miles)) +
  geom_line(linewidth = 1) +
  geom_point(size = 2) +
  scale_x_date(date_breaks = "3 months", date_labels = "%b %Y") +
  scale_y_continuous(labels = label_number_si()) +
  labs(title = "Monthly Miles Trend", x = NULL, y = NULL) +
  theme_minimal(base_size = 11) +
  theme(
    panel.grid.minor = element_blank(),
    plot.title = element_text(hjust = 0.5)
  )

# ---- Chart 2: Monthly MPG Trend ----
p_mpg <- ggplot(pdf, aes(x = month, y = average_mpg)) +
  geom_line(linewidth = 1) +
  geom_point(size = 2) +
  scale_x_date(date_breaks = "3 months", date_labels = "%b %Y") +
  scale_y_continuous(labels = label_number(accuracy = 0.1)) +
  labs(title = "Monthly MPG Trend", x = NULL, y = NULL) +
  theme_minimal(base_size = 11) +
  theme(
    panel.grid.minor = element_blank(),
    plot.title = element_text(hjust = 0.5)
  )

p_miles
p_mpg


StatementMeta(, 08eb05d1-9e74-482a-ae26-137968f91534, 12, Finished, Available, Finished)

Error: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'library'.(line 1, pos 0)

== SQL ==
library(SparkR)
^^^
library(ggplot2)
library(scales)

# 1) Pull data from your Gold table (Spark SQL)
sdf <- sql("
  SELECT
    month,
    total_miles,
    average_mpg
  FROM Gold_LH.dbo.fact_driver_monthly_metrics
WHERE t1.driver_id = 'DRV00001' AND t1.month >= DATE '2024-01-01'
  ORDER BY month
")

# 2) Convert Spark DataFrame -> R data.frame
pdf <- collect(sdf)

# 3) Ensure month is a Date type in R
pdf$month <- as.Date(pdf$month)

# ---- Chart 1: Monthly Miles Trend ----
p_miles <- ggplot(pdf, aes(x = month, y = total_miles)) +
  geom_line(linewidth = 1) +
  geom_point(size = 2) +
  scale_x_date(date_breaks = "3 months", date_labels = "%b %Y") +
  scale_y_continuous(labels = label_number_si()) +
  labs(title = "Monthly Miles Trend", x = NULL, y = NULL) +
  theme_minimal(base_size = 11) +
  theme(
    panel.grid.minor = element_blank(),
    plot.title = element_text(hjust = 0.5)
  )

# ---- Cha